# CR Project : Nanogram solver

## Présentation du Projet
Ce projet explore l’application des solveurs SAT et des problèmes de satisfaction de contraintes (CSP) pour résoudre des puzzles logiques, en particulier les **nonogrammes** (également appelés Picross ou Hanjie). L’objectif est de développer un **solveur automatisé** capable de déterminer si un nonogramme donné (2D et 3D) a une solution et, le cas échéant, de trouver cette solution efficacement.

Nous procéderons en deux phases :
1. **Résolution des nonogrammes 2D à l’aide d’un solveur SAT**
2. **Extension de la solution aux nonogrammes 3D**, en tenant compte de l’explosion combinatoire et en justifiant une approche de résolution ligne par ligne.

Les nonogrammes ont gagné en popularité grâce à divers jeux, notamment la série *Nintendo Picross 3D*, sortie pour la première fois en 2009, qui a introduit le concept de nanogrammes en trois dimensions.

---



## Partie 1 : Solveur de Nonogrammes 2D

### Définition du Problème
Un **nonogramme 2D** est constitué d’une grille où chaque ligne et chaque colonne est accompagnée d’une série de nombres. Ces nombres indiquent les blocs de cellules remplies consécutives, séparées par au moins une cellule vide.

#### Exemple d’entrée (cactus.txt) :
```
10  # Nombre de lignes
8   # Nombre de colonnes
2
22
22
222 # Exemple pour cette ligne on sait qu'il 3 blocs de 2 cases consécutives séparées par un nombre inconnu de cases vides
222
25
22
5
2
2
5
5
1
10
10
10
1
5
5
```

#### Exemple de solution (cactus_sol.txt) :
```
...XX...
...XX.XX
...XX.XX
XX.XX.XX
XX.XX.XX
\...
```

<p align="center">
  <img src="img/cactus_sol.png" alt="cactus solution" width="400"/>
</p>

### Représentation des Données
- **Grille** : Représentée sous forme d’une matrice de valeurs booléennes (`1` pour une case remplie, `0` pour une case vide).
- **Contraintes des lignes et colonnes** : Listes de nombres indiquant les blocs remplis requis.

### Modélisation en SAT
Chaque cellule `(i, j)` de la grille est représentée comme une variable booléenne. Les contraintes sont exprimées sous forme de formules logiques :
1. **Contraintes de ligne** garantissent le bon nombre et regroupement des cellules remplies.
2. **Contraintes de colonne** assurent la même chose pour les colonnes.
3. **Clauses binaires** imposent des espaces entre les blocs lorsque nécessaire.

Des solveurs SAT comme **PySAT** peuvent être utilisés pour résoudre ces contraintes.

### Implémentation
- **Analyse du format d’entrée**
- **Génération des contraintes SAT**
- **Résolution avec un solveur SAT**
- **Affichage visuel de la solution** avec Tkinter

---



## Partie 2 : Solveur de Nonogrammes 3D

### Différences avec le 2D
- Au lieu de lignes et colonnes, nous avons maintenant des **plans 3D**.
- **Trois ensembles de contraintes** (lignes, colonnes, profondeur).
- Modification des indices :
  - **Nombre simple** : Un bloc contigu.
  - **Nombre entouré d’un cercle** : Exactement deux blocs distincts.
  - **Nombre entouré d’un carré** : Trois blocs ou plus distincts.

### Justification de la Résolution Ligne par Ligne
En raison de l’explosion combinatoire, résoudre toute la grille 3D d’un coup est trop complexe. Une approche plus efficace consiste à :
1. Résoudre chaque **ligne indépendamment**.
2. Utiliser les déductions pour réduire l’espace de recherche.
3. Affiner itérativement jusqu’à obtenir une solution cohérente.

### Plan d’Implémentation
- **Structure des données** : Étendre la représentation 2D en 3D (en utilisant des tableaux NumPy ou des dictionnaires).
- **Contraintes** : Traduire les règles 3D en formules compatibles avec SAT.
- **Approche de résolution** : Implémenter un solveur SAT ligne par ligne.
- **Visualisation** : Utiliser Open3D ou Matplotlib pour l’affichage.

### Références aux Travaux Existants
Nous nous inspirons de projets existants, notamment :
- [Picross 3D Solver](https://github.com/epmjohnston/Picross-3D-Solver)
- Article de recherche : *Generating and solving 3D nonogram puzzles* (Connor Halford, 2016)

---

## Conclusion et Améliorations Possibles
Si le temps le permet, des améliorations possibles incluent :
- **Génération de puzzles** : Créer des nonogrammes solvables à partir de modèles voxelisés.
- **Solveurs optimisés** : Approches hybrides combinant SAT et heuristiques.
- **Interface utilisateur** : Expérience de résolution interactive.